<a href="https://colab.research.google.com/github/saketja/Gen-AI-Image-Colorization/blob/main/Gen_AI_Image_Colorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
  print(torch.cuda.current_device)
else:
  print("No NVIDIA driver found. Using CPU")

No NVIDIA driver found. Using CPU


In [12]:
transform = transforms.Compose([transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False,num_workers=2)

In [26]:
class ColorizationNet(nn.Module):
  def __init__(self):
    super(ColorizationNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=4, dilation=2)
    self.conv2 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=4, dilation=2)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=4, dilation=2)
    self.conv4 = nn.Conv2d(128, 3, kernel_size=5, stride=1, padding=4, dilation=2)

  def forward(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = nn.functional.relu(self.conv2(x))
    x = nn.functional.relu(self.conv3(x))
    x = torch.sigmoid(self.conv4(x))

    return x

In [27]:
model = ColorizationNet().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def rgb_to_gray(img):
  return img.mean(dim=1,keepdim=True)

In [ ]:
EPOCHS = 30
for epoch in range(EPOCHS):
                for i,(images,_) in enumerate(train_loader):
                    grayscale_images = rgb_to_gray(images).to(device)
                    images = images.to(device)
                    outputs = model(grayscale_images)
                    loss = criterion(outputs, images)


                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    if i % 100 == 0:
                        print(f"Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

print("Training finished")

Epoch [1/30], Step [1/782], Loss: 0.0232
Epoch [1/30], Step [101/782], Loss: 0.0065
Epoch [1/30], Step [201/782], Loss: 0.0063
Epoch [1/30], Step [301/782], Loss: 0.0060
Epoch [1/30], Step [401/782], Loss: 0.0049
Epoch [1/30], Step [501/782], Loss: 0.0048


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
  img = img/2 + 0.5
  npimg = img.numpy()

  if len(img.shape)==2
    plt.imshow(npimg, cmap="gray")
  else:
    plt.imshow(np.transpose(npimg, (1,2,0)))

def visualize_all_three(original_images,grayscale_images,colorizwd_images,n=5):
    fig = plt.figure(figsize=(3*n,4))
    for i in range(n):
        ax = plt.subplot(1,3*n,3*i+1)
        imshow(original_images[i])
        ax.set_title("Original")
        ax.axis("off")

        ax = plt.subplot(1,3*n,3*i+2)
        imshow(grayscale_images[i])
        ax.set_title("Grayscale")
        ax.axis("off")

        ax = plt.subplot(1,3*n,3*i+3)
        imshow(colorizwd_images[i])
        ax.set_title("Colorized")
        ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
def torch_rgb_to_hsv(rgb):
  r,g,b = rgb[:0,:,:], rgb[:,1,:,:],rgb[:,2,:,:]

  max_val, _ = torch.max(rgb,dim=1)
  min_val, _ = torch.min(rgb,dim=1)

  h = torch.zero_like(r)
  mask = (max_val==r)&(g>=b)
  h[mask] = (g[mask]-b[mask])/diff[mask]
  mask = (max_val==r)&(g<b)
  h[mask] = (g[mask]-b[mask])/diff(mask) + 6.0
  mask = (max_val==g)
  h[mask] = (b[mask]-r[mask])/diff(mask) + 2.0
  mask = (max_val==b)
  h[mask] = (r[mask]-g[mask])/diff(mask) + 4.0
  h = h / 6.0
  h[diff == 0.0] = 0.0

  s = torch.zeros_like(r)
  s[diff != 0.0] = diff[diff != 0.0]/max_val[diff !=0.0]

  v = max_val

  return torch.stack([h,s,v],dim=1)

In [ ]:
h, s, v = hsv[:, 0, :, :], hsv[:, 1, :, :], hsv[:, 2, :, :]
    i = (h * 6.0).floor()
    f = h * 6.0 - i
    p = v * (1.0 - s)
    q = v * (1.0 - s * f)
    t = v * (1.0 - s * (1.0 - f))

    i_mod = i % 6
    r = torch.zeros_like(h)
    g = torch.zeros_like(h)
    b = torch.zeros_like(h)

    r[i_mod == 0.0] = v[i_mod == 0.0]
    g[i_mod == 0.0] = t[i_mod == 0.0]
    b[i_mod == 0.0] = p[i_mod == 0.0]

    r[i_mod == 1.0] = q[i_mod == 1.0]
    g[i_mod == 1.0] = v[i_mod == 1.0]
    b[i_mod == 1.0] = p[i_mod == 1.0]

    r[i_mod == 2.0] = p[i_mod == 2.0]
    g[i_mod == 2.0] = v[i_mod == 2.0]
    b[i_mod == 2.0] = t[i_mod == 2.0]

    r[i_mod == 3.0] = p[i_mod == 3.0]
    g[i_mod == 3.0] = q[i_mod == 3.0]
    b[i_mod == 3.0] = v[i_mod == 3.0]

    r[i_mod == 4.0] = t[i_mod == 4.0]
    g[i_mod == 4.0] = p[i_mod == 4.0]
    b[i_mod == 4.0] = v[i_mod == 4.0]

    r[i_mod == 5.0] = v[i_mod == 5.0]
    g[i_mod == 5.0] = p[i_mod == 5.0]
    b[i_mod == 5.0] = q[i_mod == 5.0]

    return torch.stack([r, g, b], dim=1)

In [ ]:
def exaggerate_colors(images.saturation_factor=1.5,value_factor=1.2):
  images = (images + 1) / 2.0

    # Convert RGB images to HSV
    images_hsv = torch_rgb_to_hsv(images)

    # Increase the saturation and value components
    images_hsv[:, 1, :, :] = torch.clamp(images_hsv[:, 1, :, :] * saturation_factor, 0, 1)
    images_hsv[:, 2, :, :] = torch.clamp(images_hsv[:, 2, :, :] * value_factor, 0, 1)

    # Convert the modified HSV images back to RGB
    color_exaggerated_images = torch_hsv_to_rgb(images_hsv)

    # Convert images back to the range [-1, 1]
    color_exaggerated_images = color_exaggerated_images * 2.0 - 1.0

    return color_exaggerated_images

In [ ]:
with torch.no_grad():
    for i, (images, _) in enumerate(test_loader):
        grayscale_images = rgb_to_gray(images).to(device)
        colorized_images = model(grayscale_images)


        grayscale_images_cpu = grayscale_images.cpu().squeeze(1)
        colorized_images_cpu = colorized_images.cpu()
        original_images_cpu = images.cpu()

        #colorized_images_cpu=scale_predicted_colors(colorized_images_cpu)
        colorized_images_cpu=exaggerate_colors(colorized_images_cpu)

        # Visualize the grayscale, colorized, and original images
        visualize_all_three(original_images_cpu, grayscale_images_cpu, colorized_images_cpu)

        if i == 10:  # only do this for up to certain batch for demonstration purposes
            break

In [ ]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current working directory:", current_directory)

In [ ]:
from google.colab import files

# Upload files from your local machine
uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    print("Uploaded file:", filename)

In [ ]:
import os

# List all files in the current directory
files = os.listdir("/content")
print("Files in the current directory:", files)

In [ ]:
from PIL import Image

# Open the image. (Keep your image in the current directory. In my case, the image was horse.jpg)
img = Image.open('eiffiltower.jpeg')

# Convert the image to grayscale
gray_img = img.convert("L")

In [ ]:
import torchvision.transforms as transforms

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    # If you need to normalize, uncomment the following line
    # transforms.Normalize(mean=[0.5], std=[0.5])  # Assuming you want to normalize to [-1, 1] range
])

In [ ]:
# Apply the transformations
img_tensor = transform(gray_img).unsqueeze(0)  # Add a batch dimension

# Ensure the model is in evaluation mode
model.eval()

# Move the image tensor to the device where your model is (likely 'cuda' if using GPU)
img_tensor = img_tensor.to(device)

# Get the model's output
with torch.no_grad():
    colorized_tensor = model(img_tensor)

In [ ]:
# Convert the tensor back to an image
colorized_img = transforms.ToPILImage()(colorized_tensor.squeeze(0).cpu())

# Optionally, save the image
colorized_img.save("_colorized2.jpg")

In [ ]:
# Plotting the original, grayscale, and colorized images side-by-side
fig, ax = plt.subplots(1, 3, figsize=(18, 6))  # Create a figure with 1 row and 3 columns

# Display original color image
ax[0].imshow(img)
ax[0].set_title("Original Color Image")
ax[0].axis('off')  # Hide axes

# Display grayscale image
ax[1].imshow(gray_img, cmap='gray')  # Since it's grayscale, use cmap='gray'
ax[1].set_title("Grayscale Image")
ax[1].axis('off')  # Hide axes

# Display colorized image
ax[2].imshow(colorized_img)
ax[2].set_title("Colorized Image")
ax[2].axis('off')  # Hide axes

plt.tight_layout()  # Adjust spacing
plt.show()